<a href="https://colab.research.google.com/github/Roby290597/nlp_exercise/blob/main/bundestags_reden_analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import requests
from sentence_transformers import SentenceTransformer
import numpy as np  
import sys




ModuleNotFoundError: No module named 'redner_extraction'

In [ ]:
import sys
from pathlib import Path

extr = next((str(p/"extr") for p in [Path.cwd()] + list(Path.cwd().parents) if (p/"extr").is_dir()), None)
if extr and extr not in sys.path:
    sys.path.insert(0, extr)
print("extr hinzugefügt zu sys.path:", extr)

from redner_extraction import extract_speeches, extract_all_speakers, get_average_sentiment


### Extraction of bundestag data  

In [2]:
url = "https://www.bundestag.de/resource/blob/1115000/21032.xml" ### Bundestag Reden XML Datei vom November 2025 (Stand 30. November 2025 war es das aktuellste)
response = requests.get(url)

# XML in einen Tree parsen
tree = ET.ElementTree(ET.fromstring(response.content))

# Wurzel-Element abrufen
root = tree.getroot()



In [3]:
reden = {}
redner = []
# Beispiel-Ausgabe
for speech in extract_speeches(root):
    #print(f"Redner: {speech['name']}\nRede: {speech['text'][:200]}...\n")
    if speech['name'] not in redner:
        redner.append(speech['name'])
        reden[speech['name']] = speech['text']
    else:
        print("DOPPELT:", speech['name'])
        reden[speech['name']] += "\n Nächste Rede:" + speech['text']

DOPPELT: Mandy Eißing
DOPPELT: Peter Bohnhof
DOPPELT: Mahmut Özdemir
DOPPELT: Klaus Wiener
DOPPELT: Mirze Edis


In [4]:
redner_party = []
all_speakers = extract_all_speakers(root)
for speaker in all_speakers:
    print(speaker)
    redner_party.append((speaker['vorname'] + " " + speaker['nachname'], speaker['fraktion']))

{'vorname': 'Bärbel', 'nachname': 'Bas', 'fraktion': '', 'rolle_kurz': 'Bundesministerin BMAS'}
{'vorname': 'Hans-Jürgen', 'nachname': 'Goßner', 'fraktion': 'AfD', 'rolle_kurz': ''}
{'vorname': 'Wilfried', 'nachname': 'Oellers', 'fraktion': 'CDU/CSU', 'rolle_kurz': ''}
{'vorname': 'Ricarda', 'nachname': 'Lang', 'fraktion': 'BÜNDNIS\xa090/DIE GRÜNEN', 'rolle_kurz': ''}
{'vorname': 'Pascal', 'nachname': 'Meiser', 'fraktion': 'Die Linke', 'rolle_kurz': ''}
{'vorname': 'Dagmar', 'nachname': 'Schmidt', 'fraktion': 'SPD', 'rolle_kurz': ''}
{'vorname': 'Peter', 'nachname': 'Bohnhof', 'fraktion': 'AfD', 'rolle_kurz': ''}
{'vorname': 'Sandra', 'nachname': 'Carstensen', 'fraktion': 'CDU/CSU', 'rolle_kurz': ''}
{'vorname': 'Armin', 'nachname': 'Grau', 'fraktion': 'BÜNDNIS\xa090/DIE GRÜNEN', 'rolle_kurz': ''}
{'vorname': 'Nora', 'nachname': 'Seitz', 'fraktion': 'CDU/CSU', 'rolle_kurz': ''}
{'vorname': 'Leif-Erik', 'nachname': 'Holm', 'fraktion': 'AfD', 'rolle_kurz': ''}
{'vorname': 'Peter', 'nachn

In [5]:
from transformers import pipeline

# using pipeline API for summarization task
summarization = pipeline("summarization")


original_text = """

Sehr geehrte/r Frau/Herr [Name],

mit großem Interesse bewerbe ich mich als Bankkaufmann in Ihrem Haus. Ich möchte meine Stärke
 im kundenorientierten Arbeiten, meine Sorgfalt im Umgang mit finanziellen Themen sowie meine 
 Freude an Beratung und Vertrieb in Ihre Filiale/Ihr Team einbringen.
In meiner bisherigen Tätigkeit/ Ausbildung zum Bankkaufmann habe ich Privatkundinnen und -kunden 
im täglichen Bankgeschäft betreut – von Kontoführung und Zahlungsverkehr über Service- und 
Beratungsgespräche bis hin zur Terminvorbereitung für komplexere Finanzthemen. Dabei war mir wichtig, 
verständlich zu kommunizieren, Bedürfnisse sauber zu erfassen und Lösungen zuverlässig umzusetzen.
Besonders schätze ich die Kombination aus persönlicher Beratung, strukturierten Prozessen und 
verantwortungsvollem Arbeiten.
Ich arbeite gewissenhaft, diskret und belastbar – auch bei hohem Kundenaufkommen. 
Gleichzeitig bringe ich Teamgeist mit und lege Wert auf eine saubere Dokumentation 
sowie die Einhaltung regulatorischer Vorgaben. Den Umgang mit gängigen Bank-/Office-Systemen 
beherrsche ich sicher und arbeite mich schnell in neue Tools ein.
Gern überzeuge ich Sie in einem persönlichen Gespräch von meiner Motivation und Eignung. 
Ich freue mich über Ihre Rückmeldung.
Mit freundlichen Grüßen.
Max Mustermann
"""

summary_text = summarization(original_text)[0]['summary_text']
print("Summary:", summary_text)



from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(original_text, summary_text)
print("ROUGE Scores:", scores)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Summary:  Theodor Mustermann is a Bankkaufmann in Ihre Bank-/Office-Systemen . Theodor Gern: "Gern überzeuge ich Sie in einem persönlichen Gespräch von meiner Motivation and Eignung" Gern habe ich bringe ich Teamgeist mit und lege Wert auf eine saubere Dokumentation .
ROUGE Scores: {'rouge1': Score(precision=0.8292682926829268, recall=0.17708333333333334, fmeasure=0.2918454935622318), 'rougeL': Score(precision=0.5365853658536586, recall=0.11458333333333333, fmeasure=0.18884120171673818)}


In [7]:
summary_text = summarization(reden["Leif-Erik Holm"][1000:2000])[0]['summary_text']
print("Summary:", summary_text)

Summary:  Kanzler Merz ist vom Hoffnungsträger zum Bestatter der deutschen Wirtschaft mutiert . ‘Es ist fünf nach zwölf“, klagt die Verbandschefin Ostermann . Wieder geht es den Kleinen an den Kragen mit diesem Gesetz .


## Calcutation of the rouge score 


A description of the rouge_score is given on this website: https://spotintelligence.com/2024/08/12/rouge-metric-in-nlp/

In [ ]:
from rouge_score import rouge_scorer

# Initialisierung des Scorers 
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL', 'rougeLsum'], use_stemmer=True)


for name, text in reden.items():
    summary_text = summarization(text[:1000])[0]['summary_text']
    scores = scorer.score(text[:1000], summary_text)
    print(f"Redner: {name}\nZusammenfassung: {summary_text} ")
    print("ROUGE Scores:", scores)
    print("\n")

Redner: Bärbel Bas
Zusammenfassung:  500 Milliarden Euro – 500 million Euro – ise viel Geld in die Hand genommen, um unser Land voranzubringen . Wir investieren in Kitas, in Schulen, in Krankenhäuser, in Klimaschutz. Wir stärken die kritische Infrastruktur, auch in Zeiten neuartiger Bedrohungen . 
ROUGE Scores: {'rouge1': Score(precision=0.8974358974358975, recall=0.23333333333333334, fmeasure=0.3703703703703704), 'rougeL': Score(precision=0.8974358974358975, recall=0.23333333333333334, fmeasure=0.3703703703703704)}


Redner: Hans-Jürgen Goßner
Zusammenfassung:  Tariftreuegesetz ist wieder einmal ein Paradebeispiel deutscher Bürokratiepolitik . Wer solche Widersprüche produziert, gehört nicht in die Regierung, sondern auf die Comedybühne . 
ROUGE Scores: {'rouge1': Score(precision=1.0, recall=0.1592356687898089, fmeasure=0.2747252747252747), 'rougeL': Score(precision=1.0, recall=0.1592356687898089, fmeasure=0.2747252747252747)}


Redner: Wilfried Oellers
Zusammenfassung:  Der Gesetzent

In [ ]:
from rouge_score import rouge_scorer


senetence1 = "The quick brown fox jumps on the dog !"
senetence2 = "The quick brown fox jumps on the dog."

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=True)
scores = scorer.score(senetence1, senetence2)




from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)



from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)


prefix = "summarize: "


{'rouge1': Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8),
 'rouge2': Score(precision=1.0, recall=0.6363636363636364, fmeasure=0.7777777777777778),
 'rougeL': Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)}

In [50]:
from datasets import load_dataset


prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


billsum = load_dataset("billsum", split="ca_test")
billsum = billsum.train_test_split(test_size=0.9)
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map: 100%|██████████| 1114/1114 [00:02<00:00, 475.21 examples/s]


In [61]:
billsum = load_dataset("billsum", split="ca_test")


In [ ]:
billsum = billsum.train_test_split(test_size=0.999)

tokenized_billsum = billsum.map(preprocess_function, batched=True)

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'